## Notebook for calculating various molecular desciptors

In [8]:
# Imports
from rdkit import Chem, RDLogger
import os
from mordred import Calculator, descriptors
import pandas as pd
from more_itertools import chunked
from tqdm.notebook import tqdm
# Capturing RDKit errors
from io import StringIO
import sys
Chem.WrapLogs()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
RDLogger.DisableLog('rdApp.*')

In [3]:
def mol2_to_mol(file=None, sanitize=True):
    mols=[]
    with open(file, 'r') as f:
        line =f.readline()
        while not f.tell() == os.fstat(f.fileno()).st_size:
            if line.startswith("@<TRIPOS>MOLECULE"):
                mol = []
                mol.append(line)
                line = f.readline()
                while not line.startswith("@<TRIPOS>MOLECULE"):
                    mol.append(line)
                    line = f.readline()
                    if f.tell() == os.fstat(f.fileno()).st_size:
                        mol.append(line)
                        break
                mol[-1] = mol[-1].rstrip() # removes blank line at file end
                block = ",".join(mol).replace(',','')
                m=Chem.MolFromMol2Block(block, sanitize=sanitize)
            if m is not None:
                mols.append(m)
    return(mols)

In [19]:
mols = mol2_to_mol('../small_molecule_search.mol2')

In [ ]:
def number_molecules(mols):
    # Adds number to the molecule as an identifier. 
    # This will be written in the .mol2 file.
    for i, mol in enumerate(mols):
        mol.SetProp('_Name', str(i))
    return mols

def parse_molecules(mols):
    # Removes molecules that generate warnings when parsing through RDKit
    # This could lead to incorrect values depending on how the descriptor
    # is calculated.
    working_mols = []
    nonworking_mols = []
    sio = sys.stderr = StringIO()
    for mol in mols:
        Chem.SanitizeMol(mol)
        res = sio.getvalue()
        if 'WARNING' in res:
            nonworking_mols.append(mol)
            print(res)
            # Reset stderr
            sio = sys.stderr = StringIO()
        else:
            working_mols.append(mol)
            sio = sys.stderr = StringIO()
    working_mols = number_molecules(working_mols)
    return working_mols, nonworking_mols

def calculate_descriptors_pandas(mols, check_mols=False):
    # Calculates 2D and 3D descriptors using Mordred.
    # Returns a DataFrame containing descriptors.
    calc = Calculator(descriptors, ignore_3D=False)
    if check_mols:
        mols, _ = parse_molecules(mols)
    df = calc.pandas(working_mols, ipynb=True, quiet=False)
    df.index.name = 'Molecule_Number'
    df['SMILES'] = [Chem.MolToSmiles(m) for m in mols]
    df['RDKit_Molecule'] = mols
    return df

In [ ]:
df.to_pickle('Calculated_Descriptors.pkl')